# Preparation

## Import required Pythmon packages

In [1]:
import numpy as np
import pandas as pd

In [2]:
from datetime import date
from datetime import datetime, timedelta
import time
import pytz
import glob

In [3]:
#%load_ext autotime
#%load_ext pycodestyle_magic
#%flake8_on

## Import data

In [4]:
# Network locations for data import & export
input_TravixDatadump_BI = r'\\192.168.140.37\\Business-Share\\bi-report\\Data analysis resources\\Travix\\DatadumpTravix\\'
Hepstar_path =r'\\192.168.140.37\\Business-Share\\bi-report\\'
input_BI_drive = r'\\192.168.140.37\Business-Share\BI resources\Dashboards\Travix\# Data\\'
export_BI_drive = r'\\192.168.140.37\Business-Share\BI resources\Dashboards\Travix\# Data\\'

In [5]:
 # import HS clean data
HS_df1 = pd.read_csv(input_BI_drive+'20190701-present HS_clean_df.csv', error_bad_lines=False,low_memory=False)
HS_df2 = pd.read_csv(input_BI_drive+'20180501-20190630 HS_clean_df.csv', error_bad_lines=False,low_memory=False)

In [6]:
# Merge Travix csv files in folders with other csv files not in folders
HS_df = pd.concat([HS_df1, HS_df2], sort = False)

In [52]:
# make a copy of the dataset
HS_base_df = HS_df.copy()
HS_base_df = HS_base_df.copy()

# Data cleaning

In [85]:
#Select relevant data
HS_T_df=HS_base_df.loc[HS_base_df['channel'].isin(['CheapticketsNL','BudgetAir','Vliegwinkel'])]
#make a copy of the data set
HS_T_df= HS_T_df.copy()

In [54]:
# Change date variable
HS_T_df['date'] = pd.to_datetime(HS_T_df['date'])

In [55]:
#Select relevant data
HS_T_df = HS_T_df[(HS_T_df['date']>='2019-01-01')& (HS_T_df['date']<='2019-12-31')]

In [58]:
#Calculate 50% premium_distributor revenue 
def premium_distributor (HS_T_df):
    if (HS_T_df['supplier'] == 'Unigarant'):
        return  HS_T_df['premium_sum'] * 0.58
    if (HS_T_df['supplier'] == 'Refund Protect'):
        return  HS_T_df['premium_sum'] * 0.50
# customers who were not offered any products
    if (HS_T_df['supplier'] == ''):
        return  0
    else:
        return 0
HS_T_df['premium_distr_revenue'] = HS_T_df.apply (lambda row: premium_distributor (row), axis=1)

In [60]:
HS_T_df['premium_distr_revenue'] = HS_T_df['premium_distr_revenue'].astype(int)

In [61]:
#Add a column that indicates the quater of the year in which the order_date is in
HS_T_df['quarter'] = pd.PeriodIndex(HS_T_df['date'],freq='Q')

In [62]:
# Calculate the number of days to departure
HS_T_df['startdate'] = pd.to_datetime(HS_T_df['startdate'])
HS_T_df['enddate'] = pd.to_datetime(HS_T_df['enddate'])
HS_T_df['daystodep']=HS_T_df['startdate']-HS_T_df['date']

In [63]:
#converting number of days to integer type
HS_T_df['daystodep']=HS_T_df['daystodep'].dt.days

In [64]:
# Drop if startdate is before booking date
HS_T_df.drop(HS_T_df[HS_T_df['date']>HS_T_df['startdate']].index, inplace=True)

In [65]:
#calculate  to departure bracket
def Travix_depature (HS_T_df):
    if (HS_T_df['daystodep'] == 0):
        return 'b_0~0'
    if (HS_T_df['daystodep'] == 1):
        return 'c_1~1'
    if (HS_T_df['daystodep'] > 1 ) and (HS_T_df['daystodep'] <=10):
        return 'd_2~10'
    if (HS_T_df['daystodep'] > 10 ) and (HS_T_df['daystodep'] <=20):
        return 'a_11~20'
    if (HS_T_df['daystodep'] > 20 ) and (HS_T_df['daystodep'] <=30):
        return 'e_21~30'
    if (HS_T_df['daystodep'] > 30 ) and (HS_T_df['daystodep'] <=40):
        return 'f_31~40'
    if (HS_T_df['daystodep'] > 40 ) and (HS_T_df['daystodep'] <=50):
        return 'g_41~50'
    if (HS_T_df['daystodep'] > 50 ) and (HS_T_df['daystodep'] <=60):
        return 'h_51~60'
    if (HS_T_df['daystodep'] > 60 ) and (HS_T_df['daystodep'] <=70):
        return 'i_61~70'
    if (HS_T_df['daystodep'] > 70 ) and (HS_T_df['daystodep'] <=80):
        return 'j_71~80'
    if (HS_T_df['daystodep'] > 80 ) and (HS_T_df['daystodep'] <=90):
        return 'k_81~90'
    if (HS_T_df['daystodep'] > 90 ) and (HS_T_df['daystodep'] <=100):
        return 'l_91~100'
    if (HS_T_df['daystodep'] > 100 ) and (HS_T_df['daystodep'] <=140):
        return 'm_101~140'
    if (HS_T_df['daystodep'] > 140 ):
        return 'n_>140'
    else:
        return 'o_missing'
HS_T_df['Daystodep_brac'] = HS_T_df.apply (lambda row: Travix_depature (row), axis=1)
HS_T_df['Daystodep_brac'].unique()

array(['d_2~10', 'c_1~1', 'j_71~80', 'g_41~50', 'f_31~40', 'h_51~60',
       'a_11~20', 'b_0~0', 'm_101~140', 'e_21~30', 'k_81~90', 'i_61~70',
       'n_>140', 'l_91~100'], dtype=object)

In [78]:
# select relevant data
HS_T_df = HS_T_df[['Order_month','quarter','Order_year','channel','triptype','taglocale','tagcabinclass','tagticketservice',
  'tagfrequentflyer','tagtype','tagscope','HS_cancelled','destination_region','Destination subregion','Origin region',
'Origin subregion','World_Europe','Dist_vs_Hepstar','supplier','Phase_widget_segment','splitcode','tagresidency',
  'product','Daystodep_brac','AirhelpPlusMargin','HS_purchase','HS_purchased_dist_revenue','Insured_passengers','Insurer_child_count',
  'Main_products_count','NumberOfChildren','Offered_HS_product','Pricecalls','ServicePackageMargin',
  'TicketGuaranteeMargin','Widget_product_segment','bookingvalue','claims provision','daystodep','distributor claim provision revenue',
  'distributor_purchase','masterpolicyfee_ex_tax&HS','masterpolicyfeetotal','masterpolicyfeenet','masterpolicyfeetax',
'numberoflinkedproducts','numberofpassengers','numberofpolicyissuecalls','numberofpricedcalls','numberofsegments',
'premium_sum','product_type_No insurance','tagbookingmonth','tagtravelmonth',
  'tax_sum','totalcost_sum','travelitemvalue','premium_distr_revenue']]

In [80]:
#Check if there is any  missing data
HS_T_df.isnull().sum()

Order_month                            0
quarter                                0
Order_year                             0
channel                                0
triptype                               0
taglocale                              0
tagcabinclass                          0
tagticketservice                       0
tagfrequentflyer                       0
tagtype                                0
tagscope                               0
HS_cancelled                           0
destination_region                     0
Destination subregion                  0
Origin region                          0
Origin subregion                       0
World_Europe                           0
Dist_vs_Hepstar                        0
supplier                               0
Phase_widget_segment                   0
splitcode                              0
tagresidency                           0
product                                0
Daystodep_brac                         0
AirhelpPlusMargi

In [86]:
#Fill missing values with n/a
HS_T_df.fillna(0, inplace=True)

In [87]:
#Check if there are still any missing values
HS_T_df.isnull().sum()

Order_month                            0
quarter                                0
Order_year                             0
channel                                0
triptype                               0
taglocale                              0
tagcabinclass                          0
tagticketservice                       0
tagfrequentflyer                       0
tagtype                                0
tagscope                               0
HS_cancelled                           0
destination_region                     0
Destination subregion                  0
Origin region                          0
Origin subregion                       0
World_Europe                           0
Dist_vs_Hepstar                        0
supplier                               0
Phase_widget_segment                   0
splitcode                              0
tagresidency                           0
product                                0
Daystodep_brac                         0
AirhelpPlusMargi

In [88]:
#rename distributor to control
HS_T_df['Dist_vs_Hepstar'].replace('Distributor', 'Control',inplace=True)

In [89]:
#Create a pivot table that groups data per year, week, haul and policy type
HS_T_pv_df=HS_T_df.pivot_table(index=['Order_month','quarter','Order_year','channel','triptype','taglocale','tagcabinclass','tagticketservice',
  'tagfrequentflyer','tagtype','tagscope','HS_cancelled','destination_region','Destination subregion','Origin region',
'Origin subregion','World_Europe','Dist_vs_Hepstar','supplier','Phase_widget_segment','splitcode','tagresidency',
  'product','Daystodep_brac'], aggfunc='sum')

In [90]:
HS_T_pv_df.reset_index(inplace=True)

# Export data

In [91]:
HS_T_pv_df.to_csv(export_BI_drive+'Quartely Board report data.csv')